In [326]:
import pandas as pd
from random import sample
import tweepy
import pickle

In [561]:
df_anno = pd.read_csv(r'anno_final.csv')
df_hbm_only = pd.read_csv('hbm_only.csv')

In [3]:
df_anno_hbm_only = df_anno[df_anno.HBM_related == 1]

In [4]:
df_hbm_only_combined = pd.concat([df_anno_hbm_only, df_hbm_only])

In [5]:
df_sdoh = pd.read_csv('us_only.csv')

In [6]:
df_hbm_us_only = df_hbm_only_combined[df_hbm_only_combined.tweet_id.isin(df_sdoh.tweet_id)]

In [10]:
data_needed = list(set([int(x) for x in df_hbm_us_only[~(df_hbm_us_only.author_id.isin(list(set(data_combined.keys()))))]['author_id']]))

In [12]:
consumer_key = ""
consumer_secret = ""
access_token = ""
access_token_secret = ""

In [13]:
# authorization of consumer key and consumer secret
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
 
# set access to user's access key and access secret
auth.set_access_token(access_token, access_token_secret)
 
# calling the api
api = tweepy.API(auth)

In [14]:
user_info = {}

In [15]:
for x in data_needed:
    try:
        user = api.get_user(x)
        user_info.update({x: user})
    except:
        user_info.update({x: "not_found"})

In [562]:
df_hbm_us_only['user_profile'] = df_hbm_us_only['author_id'].map(user_info)

In [19]:
#have the user data individually

In [20]:
user_data_found = df_hbm_us_only[df_hbm_us_only.user_profile != 'not_found']

In [21]:
user_data_found['user_profile_data'] = user_data_found.user_profile.apply(lambda x: x._json)

C:\Users\chaewon7\AppData\Local\Temp\ipykernel_12992\895509049.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_data_found['user_profile_data'] = user_data_found.user_profile.apply(lambda x: x._json)


In [22]:
user_data_found_list = []

In [23]:
for x in range(0, len(user_data_found)):
    user_data_found_list.append(user_data_found.iloc[x]['user_profile_data'])

In [24]:
df_user_data_found = pd.DataFrame(user_data_found_list)

In [25]:
df_user_data_found.to_csv('user_profile_full.csv', index=False)

In [46]:
# address_concat

In [73]:
location_dict = dict(zip(df_hbm_us_only.author_id, df_hbm_us_only.location))
address_dict = dict(zip(df_hbm_us_only.author_id, df_hbm_us_only.address_concat))

In [74]:
df_user_data_found['location_from_data'] = df_user_data_found['id'].map(location_dict)
df_user_data_found['geo_from_data'] = df_user_data_found['id'].map(address_dict)

In [77]:
df_user_data_found = df_user_data_found[['id', 'id_str', 'name', 'screen_name', 'location', 'location_from_data',
       'geo_from_data', 'profile_location', 'description', 'url', 'entities', 'protected', 'followers_count',
       'friends_count', 'listed_count', 'created_at', 'favourites_count',
       'utc_offset', 'time_zone', 'geo_enabled', 'verified', 'statuses_count',
       'lang', 'status', 'contributors_enabled', 'is_translator',
       'is_translation_enabled', 'profile_background_color',
       'profile_background_image_url', 'profile_background_image_url_https',
       'profile_background_tile', 'profile_image_url',
       'profile_image_url_https', 'profile_link_color',
       'profile_sidebar_border_color', 'profile_sidebar_fill_color',
       'profile_text_color', 'profile_use_background_image',
       'has_extended_profile', 'default_profile', 'default_profile_image',
       'following', 'follow_request_sent', 'notifications', 'translator_type',
       'withheld_in_countries', 'profile_banner_url']]

In [79]:
df_user_data_found[(df_user_data_found.location == '') & (df_user_data_found.location_from_data == '') & (df_user_data_found.geo_from_data.isnull())]

,id,id_str,name,screen_name,location,location_from_data,geo_from_data,profile_location,description,url,...,profile_use_background_image,has_extended_profile,default_profile,default_profile_image,following,follow_request_sent,notifications,translator_type,withheld_in_countries,profile_banner_url


In [288]:
df_user_data_found_no_dup = df_user_data_found[~(df_user_data_found.duplicated(['id_str']))]

In [289]:
df_user_data_found_no_dup.to_csv('user_profile_full.csv', index=False)